In [101]:
from functools import reduce
from sympy import (
    I,
    symbols,
    sqrt,
    Rational,
    conjugate,
    simplify,
    Poly,
)


# Notes

## Preliminaries

Rotation (in three dimensions) of the vector ${\bf r}$ about the direction $\hat{\bf n}$ by an angle $\theta$:

$$
{\bf r}' = {\bf r}\,\cos\theta + ({\bf r}\cdot\hat{\bf n}) \hat{\bf n}\, (1 - \cos\theta) - {\bf r} \times \hat{\bf n} \, \sin\theta
$$

or, in components,

$$
\begin{align*}
r'^i &= \left[{\delta^i}_j\,\cos\theta + n^in_j\,(1-\cos\theta)-{\epsilon^i}_{jk}\,n^k\,\sin\theta\right] r^j \\
:&= {\cal O^i}_j\,r^j \,.
\end{align*}
$$

Introducing the variables 

$$
a_i = n_i\,\sin\left(\theta/2\right) \,,\quad d:= \cos\left(\theta /2\right) \,,
$$

we see the rotation matrix ${{\cal O}^i}_j$ can be rewritten as follows:

$$
{{\cal O}^i}_j = \left(2\,d^2 -1\right){\delta^i}_j+ 2\left( a^i\,a_j - {\epsilon^i}_{jk}\,a^k\,d\right) \,.
$$
(Recall that $\cos\theta = 2\,\cos^2\left(\theta/2\right) -1 = 1-2\,\sin^2\left(\theta/2\right)$ and that $\sin\theta = 2\,\sin\left(\theta/2\right)\,\cos\left(\theta/2\right)$.)

The orthogonality of ${\cal O}$ can be verified by computing ${\cal O_k}^i\,{\cal O^k}_j$ using the expression above, which gives

$$
{({\cal O}^T {\cal O})^i}_j = {\delta^i}_j + 4\left(d^2 + {\bf a}^2 -1\right)\left(d^2 \,{\delta^i}_j + 
a^ia_j\right) = {\delta^i}_j
$$
where the second equality follows from the definition of ${\bf a}$ and $d$, along with the trig identity $\sin^2\left(\theta/2\right) + \cos^2\left(\theta/2\right) = 1$ which enforces the constraint

$$
{\bf a}^2 + d^2 = 1 \,.
$$

## Stereographic Projection

In what follows, it's helpful to map the rotations on the sphere to the complex
plane, by performing an equatorial spherical projection from the point where
$\hat{\bf n} = (0, 0, 1)$ down onto the complex plane. Denoting a general point
on the unit sphere as $\hat{\bf r} = (\xi,\eta,\zeta)$ in the general case, and
writing $z = x + iy$ with real and imaginary axes parallel to the $(\xi \eta)$
plane, we find (using similar triangles) that

$$
z = x + i\, y = \frac{\xi +i\,\eta}{1-\zeta} \,.
$$

Inverting, we find

$$ 

\xi = \frac{z + \bar{z}}r \,,\quad \eta = \frac{i(\bar{z} - z)}r \,,\quad
\zeta = \frac{z\bar{z} -1}r 

$$

where we define $r := 1 + z\bar{z}$.


Rotations about the $\zeta$-axis are particularly simple in this representation,
as they correspond to multiplying $z$ by a pure phase:

$$ 

z \to z' = e^{i\theta} \, z \quad\leftrightarrow\quad \begin{array}{cc}\xi' &=
\xi\,\cos\theta
- \eta\,\sin\theta\\
\eta' &= \xi\,\sin\theta + \eta\,\cos\theta
\end{array}\,. 

$$

If we wish to write an arbitrary rotation of a point on the sphere about some
direction specified by $\hat{\bf n}$, $\theta$ (or, equivalently, ${\bf a}$,
$d$), we can leverage the simplicity of the above by noting that the rotation
leaves the antipodal points $z_1=0$, $z_2=\infty$ fixed, and that a more general
rotation about the direction $\hat{\bf n}$ should do the same for the points

$$

z_1 = \frac{n_1+i\,n_2}{1-n_3} \,,\quad z_2 = \frac{-n_1-i\,n_2}{1+n_3} \,,

$$

which satisfy the antipodal relation $z_1\,\bar z_2 = -1$, which can be used to
demonstrate the following other useful relations:

$$

\tfrac{z_1 + z_2}{z_1 - z_2} = n_3 \,,\quad \tfrac{2 z_1 z_2}{z_2 - z_1} = n_1 +
i\,n_2 \,,\quad \frac2{z_1 - z_2} = n_1 -i\,n_2\,.

$$

In [2]:
n1, n2, n3 = symbols("n1 n2 n3", real=True)
z1 = (n1 + I * n2) / (1 - n3)
z2 = (-n1 - I * n2) / (1 + n3)

In [3]:
simplify((z1 * conjugate(z2)).subs(n3, sqrt(1 - n1**2 - n2**2)))

-1

In [4]:
simplify((z1 + z2) / (z1 - z2))

n3

In [5]:
simplify((2 * z1 * z2) / (z2 - z1))

n1 + I*n2

In [6]:
simplify((2 / (z1 - z2)).subs(n3, sqrt(1 - n1**2 - n2**2)))

n1 - I*n2

As such, the rotation leaving such points fixed is

$$

\frac{z'-z_2}{z'-z_1} = e^{i\theta} \,\frac{z-z_2}{z-z_1} \,.

$$

Solving for $z'$, the above general rotation can be expressed as

$$
\begin{align*} 

z \to z' &= \frac{z\left[(z_1 - z_2)\cos(\theta/2) + i
(z_1+z_2)\,\sin(\theta/2)\right]-2i\sin(\theta/2)z_1 z_2}{2z\,i\sin(\theta/2) +
\left[(z_1 - z_2)\cos(\theta/2)-i(z_1+z_2)\sin(\theta/2)\right]} \\
&= \frac{(d+i\,a_3)z - (a_2-i\,a_1)}{(a_2+i\,a_1)z + (d-i\,a_3)}

\end{align*}

$$

where we make use of the notation introduced above which interrelates
$\{\hat{\bf n}, \theta\}$ with $\{{\bf a}, d\}$. (The above can be readily
checked by substituting $\hat {\bf r}' =(\xi',\eta',\zeta') = {\cal O} \cdot
\hat{\bf r}$ into the expression $z' = (\xi' + i\,\eta')/(1-\zeta')$.)

## Fractionalized Coordinates

Lastly, to re-interpret the above transformation in the complex plane as a
homogeneous one, it is conventional to write $z$ in terms of its numerator and
denominator in projective coordinates:

$$

z = \frac uv \,.

$$

In terms of $u$, $v$, the transformation above is given by

$$

\begin{align*}
u' &= (d+i\,a_3)u - (a_2-i\,a_1)v \\
v' &= (a_2+i\,a_1)u + (d-i\,a_3) v \,.
\end{align*}

$$

In this space, the rotations described above are a subgroup of $\mathbb{GL}(2,
\mathbb{C})$ where the operator 

$$

{\cal O} = \begin{pmatrix} A & B \\ C & D\end{pmatrix} \quad\textrm{such that}\quad
\begin{pmatrix}u' \\ v'\end{pmatrix} = \cal O \begin{pmatrix}u \\ v\end{pmatrix}

$$

satisfies ${\rm det}\, {\cal O} = 1$, $D = A^*$, $B = -C^*$. The inverse of such
a transformation is given by

$$

{\cal O}^{-1} = \begin{pmatrix} D & -B \\ -C & A\end{pmatrix} \,.

$$

## Tetrahedral Symmetry

A tetrahedron is formed by joining alternating vertices as one traverses the
edges of a cube. Taking these four vertices to be 

$$

\begin{align*}
{\bf V}_1 &= \frac1{\sqrt{3}}(1, 1, 1) \\
{\bf V}_2 &= \frac1{\sqrt{3}}(-1, -1, 1) \\
{\bf V}_3 &= \frac1{\sqrt{3}}(1, -1, -1) \\
{\bf V}_4 &= \frac1{\sqrt{3}}(-1, 1, -1) \,,
\end{align*}
$$

(such that they each lie on the unit sphere) we find that the corresponding
spherically-projected coordinates are

$$

k_1 = -k_2 = \frac{1+i}{\sqrt{3} -1} \,,\quad k_3 = -k_4 =
\frac{1-i}{\sqrt{3}+1} \,.

$$

(Note that an equivalent tetrahedron could be constructed using the points
antipodal to the ones above; this is referred to as the "diametral tetrahedron".)

The discrete rotational symmetries of the tetrahedron can be constructed by
composing the two-fold rotation about the $z$ axis (which maps ${\bf V}_1
\leftrightarrow{\bf V}_2$, ${\bf V}_3 \leftrightarrow{\bf V}_4$) with the
three-fold rotation about the ${\bf V}_1$ axis (which maps ${\bf V}_2
\rightarrow {\bf V}_3$, ${\bf V}_3 \rightarrow {\bf V}_4$, ${\bf V}_4
\rightarrow {\bf V}_2$). On the complex plane, these correspond to

$$

\begin{align*}
I:\quad z \to z' &= -z \\
II:\quad z \to z' &= \frac{z+i}{z-i} \,,
\end{align*}

$$

respectively.

In [7]:
sp1 = (1 + I) / (sqrt(3) - 1)
sp2 = -(1 + I) / (sqrt(3) - 1)
sp3 = (1 - I) / (sqrt(3) + 1)
sp4 = (-1 + I) / (sqrt(3) + 1)

print((-sp1 - sp2).radsimp().simplify())
print((-sp3 - sp4).radsimp().simplify())
print("-" * 10)
print(((sp1 + I) / (sp1 - I) - sp1).radsimp().simplify())
print(((sp2 + I) / (sp2 - I) - sp3).radsimp().simplify())
print(((sp3 + I) / (sp3 - I) - sp4).radsimp().simplify())
print(((sp4 + I) / (sp4 - I) - sp2).radsimp().simplify())

0
0
----------
0
0
0
0


In terms of the homogeneous coordinates $u$ and $v$, these two transformations
map directly to

$$

\begin{align*}
I:&\quad \begin{array}{ccc}
u' &=& i\,u \\
v' &=& -i\,v 
\end{array} \\
II:&\quad \begin{array}{ccc}
 u' &=& \tfrac12(1+i)\,u - \tfrac12(1-i)\,v \\
 v' &=& \tfrac12 (1+i) u + \tfrac12(1-i)\,v \,.
 \end{array}
\end{align*}

% \begin{align*}
% u' &= (d+i\,a_3)u - (a_2-i\,a_1)v \\
% v' &= (a_2+i\,a_1)u + (d-i\,a_3) v \,.
% \end{align*}
$$

# Polynomial Invariants

Keeping in mind that all symmetry transformations of the tetrahedron merely
exchange a number of vertices with one another, it's expected that a polynomial
with roots at the locations of the vertices in the complex plane will be
invariant under such transformations. To see this, consider the product

$$

\begin{align*}
\Phi(u, v) &:= \prod_{i=1}^4 \left(u - k_i\, v\right) = (u^2-k_1^2)(v^2-k_3^2) \\
&= u^4 - 2i\sqrt{3}\,u^2v^2 + v^4
\end{align*}

$$

and apply the transformation $(u, v) \to (u', v')$ as before. In the general
case, this gives 

$$

\begin{align*}
\Phi(u', v') &= \prod_{i=1}^4 \left(u' - k_i\, v'\right) \\
&= \prod_{i=1}^4 \left(A - k_i\, C\right) \prod_{i=1}^4 \left(u - k_i'\, v\right) \\
&= \prod_{i=1}^4 \left(A - k_i\, C\right) \, \Phi(u, v)
\end{align*}

$$

where we define

$$

k'_i := \frac{k_i\, D - B}{- k_i\, C + A} \quad\leftrightarrow\quad k_i =
\frac{k_i'\,A+B}{k_i'\,C+D} \,.

$$

In the case of both transformations $I$ and $II$, the product $\prod_{i=1}^4 \left(u - k_i'\, v\right)$ is clearly invariant, because the vertices all transform into one another under any of the transformations. For transformation $I$, the product $\prod_{i=1}^4 \left(A - k_i\, C\right) = 1$ because $A=i$ and $C=0$. For transformation $II$, this product instead gives

$$

\begin{align*}
\prod_{i=1}^4 \left(A - k_i\, C\right) &= \left[\tfrac12 (1+i)\right]^4\prod_{i=1}^4 \left(1 - k_i\right) \\
&= -\tfrac14(1-k_1^2)(1-k_3^2) \\
&=-\tfrac14\left[1-i(2+\sqrt{3})\right]\left[1+i(2-\sqrt{3})\right] \\
&=-\frac12+i\frac{\sqrt{3}}2 = e^{2\pi i/3} \,.
\end{align*}

$$

The above guarantees that the function $\Phi^3(u,v)$ is an absolute invariant under the symmetries of the tetrahedral group.

In [8]:
sp1 = (1 + I) / (sqrt(3) - 1)
sp2 = (1 - I) / (sqrt(3) + 1)
sp3 = (-1 + I) / (sqrt(3) + 1)
sp4 = -(1 + I) / (sqrt(3) - 1)

simplify(
    (Rational(1, 2) * (1 + I)) ** 4 * (1 - sp1) * (1 - sp2) * (1 - sp3) * (1 - sp4)
)

-1/2 + sqrt(3)*I/2

The above can also be done for the edge midpoints, and face centers, of the tetrahedron.
In the latter case, it's as simple as trading $k_i \to -1/\bar k_i$ in the above expression for $\Phi$ (since the face centers are in line with the antipodes of the vertices). This yields

$$

\begin{align*}
\Psi(u, v) &:= \prod_{i=1}^4\left(u + \bar k_i^{-1}\, v\right) = \prod_{i=1}^4\left(u - \bar k_i\, v\right) \\
&= u^4 + 2i\sqrt{3}\,u^2v^2 + v^4 \,.
\end{align*}

$$

(The second-last equality follows since $k_1k_4 = k_2k_3 =-1$.) As one might expect, under transformation $II$ the function $\Psi(u,v)$ becomes

$$

\Psi(u',v') = e^{-2\pi i/3} \, \Psi(u,v) \,.

$$

Therefore, the combinations $\Phi \Psi$ and $\Psi^3$ are also absolute invariants.

In contrast, the six midpoints are at $z=0$, $z=\infty$, $z=\pm 1$, and $z=\pm i$. In this case, the invariant polynomial is given by

$$

t(u, v) = uv(u-v)(u+v)(u-iv)(u+iv) = uv(u^4-v^4) \,.

$$

Under the transformation $II$ (with $A=\tfrac12(1+i)=-B^*=C=D^*$), we see that 

$$

\begin{align*}
t(u',v') &= \left(A\,u-A^*\,v\right)\left(A\,u+A^*\,v\right) \\
&\times\left[\left(A\,u-A^*\,v\right)^2 - \left(A\,u+A^*\,v\right)^2\right]\left[\left(A\,u-A^*\,v\right)^2+\left(A\,u+A^*\,v\right)^2\right] \\
&=\left(A^2\,u^2 - A^{*2}\,v^2\right)\left[4\,AA^*\,uv\right]\left[2\left(A^2\,u^2+A^{*2}\,v^2\right)\right] \\
&=\frac i2\left(u^2 + v^2\right)\left[-2\,uv\right]\left[i\left(u^2-v^2\right)\right] = t(u,v)
\end{align*}

$$

where, in the last line, we make use of $A^2 = i/2 = -A^{*2}$. Similarly, $t(u',v') = t(u,v)$ under transformation $I$ where $A=i=D^*$, $B=C=0$, we find

$$

\begin{align*}
t(u',v') &= AA^*\,uv(A^2\,u^2-A^{*2}\,v^2)(A^2\,u^2+A^{*2}\,v^2) \\
&= t(u,v) \,.
\end{align*}

$$

Therefore, $t(u,v)$ is an absolute invariant as well.

By direct computation, one can evaluate the combination $\Psi^3- \Phi^3$. As it
turns out, this expression has a particularly simple form:

$$

\Psi^3(u, v) = \Phi^3(u, v) + 12\sqrt{3}\,i\,t^2(u, v) \,.

$$

In [9]:
from tetra import Phi, Psi

simplify(Psi**3 - Phi**3).factor()

uv_prime=Matrix([
[u*(1/2 + I/2) + v*(-1/2 + I/2)],
[ u*(1/2 + I/2) + v*(1/2 - I/2)]])
u**4 - 2*sqrt(3)*I*u**2*v**2 + v**4
u**4 - 2*sqrt(3)*I*u**2*v**2 + v**4
u**12 - 6*sqrt(3)*I*u**10*v**2 - 33*u**8*v**4 + 12*sqrt(3)*I*u**6*v**6 - 33*u**4*v**8 - 6*sqrt(3)*I*u**2*v**10 + v**12
u**12 - 6*sqrt(3)*I*u**10*v**2 - 33*u**8*v**4 + 12*sqrt(3)*I*u**6*v**6 - 33*u**4*v**8 - 6*sqrt(3)*I*u**2*v**10 + v**12
u**4 + 2*sqrt(3)*I*u**2*v**2 + v**4
u**4 + 2*sqrt(3)*I*u**2*v**2 + v**4
u**12 + 6*sqrt(3)*I*u**10*v**2 - 33*u**8*v**4 - 12*sqrt(3)*I*u**6*v**6 - 33*u**4*v**8 + 6*sqrt(3)*I*u**2*v**10 + v**12
u**12 + 6*sqrt(3)*I*u**10*v**2 - 33*u**8*v**4 - 12*sqrt(3)*I*u**6*v**6 - 33*u**4*v**8 + 6*sqrt(3)*I*u**2*v**10 + v**12


12*sqrt(3)*I*u**2*v**2*(u - v)**2*(u + v)**2*(u**2 + v**2)**2

In [10]:
u, v = symbols("u v")
t = u * v * (u - v) * (u + v) * (u**2 + v**2)
simplify(Psi**3 - Phi**3 - 12 * sqrt(3) * I * t**2)

0

## Octahedral Symmetry

The six vertices of the octahedron can be taken as identical to the six edge
midpoints of the tetrahedron. As such they have $t(u,v)$ as their polynomial
invariant.

The octahedral symmetry group is generated by transformations $I$ and $II$
above, along with

$$
III: \quad u'=e^{i\pi/4}\, u \,,\quad v' = e^{-i\pi/4} \,v \,.
$$

Under transformation $III$ with $A=e^{i\pi/4} = D^*$, $B=C=0$, we find  that

$$

\begin{align*}
t(u',v') &= AA^*\,uv(A^2\,u^2-A^{*2}\,v^2)(A^2\,u^2+A^{*2}\,v^2) \\
&= -t(u,v) \,.
\end{align*}

$$

Therefore, only $t^2(u,v)$ is an absolute invariant under the octahedral
symmetry group.

The 8  face centers of the octahedron are formed by combining the 8 tetrahedral
roots $\{k_i, \bar{k}_i\}$ found before. As such, the octahedral polynomial
invariant for the face centers is given by

$$

W(u, v) := \Phi(u,v)\, \Psi(u,v)  = u^8 + 14\,u^4v^4+v^8\,.

$$

In [11]:
W = simplify(Phi * Psi)
W

u**8 + 14*u**4*v**4 + v**8

In order to work out the transformation properties of $W(u,v)$, it is useful to
note that $t$ and $W$ are inter-related as follows:

$$

\det\left[\begin{matrix}\frac{\partial^2t}{\partial u^2} & \frac{\partial^2t}{\partial u\partial v} \\[6pt] \frac{\partial^2t}{\partial v \partial u} & \frac{\partial^2t}{\partial v^2}\end{matrix}\right] = -25\,W(u,v)\,.

$$

In other words, $W(u,v)$ is the *Hessian* of $t(u,v)$. As such, since $t \to -t$
under transformation $III$ (and since $W \sim t^2$), $W(u,v)$ is clearly
invariant under all three octahedral symmetry generators.

In [12]:
from sympy import diff

du2 = diff(diff(t, u), u)
dudv = diff(diff(t, u), v)
dv2 = diff(diff(t, v), v)

simplify(du2 * dv2 - dudv**2 + 25 * Phi * Psi)

0

The edge midpoints of the octahedron form a polynomial invariant, $\chi(u,v)$,
which we will compute next.

The edge midpoints can be labeled as follows:

$$

\begin{alignat*}{3}
{\bf X}_1 &= \tfrac1{\sqrt{2}}\left(1,1, 0\right) \qquad &&{\bf X}_7 &&= \tfrac1{\sqrt{2}}\left(0,-1, 1\right)\\
{\bf X}_2 &= \tfrac1{\sqrt{2}}\left(1,-1, 0\right) \qquad&&{\bf X}_8 &&= \tfrac1{\sqrt{2}}\left(0,-1, -1\right)\\
{\bf X}_3 &= \tfrac1{\sqrt{2}}\left(-1,1, 0\right) \qquad&&{\bf X}_9 &&= \tfrac1{\sqrt{2}}\left(1,0, 1\right)\\
{\bf X}_4 &= \tfrac1{\sqrt{2}}\left(-1,-1, 0\right) \qquad&&{\bf X}_{10} &&= \tfrac1{\sqrt{2}}\left(1,0, -1\right)\\
{\bf X}_5 &= \tfrac1{\sqrt{2}}\left(0,1, 1\right) \qquad&&{\bf X}_{11} &&= \tfrac1{\sqrt{2}}\left(-1,0, 1\right)\\
{\bf X}_6 &= \tfrac1{\sqrt{2}}\left(0,1, -1\right) \qquad&&{\bf X}_{12} &&= \tfrac1{\sqrt{2}}\left(-1,0, -1\right)\\

\end{alignat*}

$$

In [13]:
def stereo(v1, v2, v3):
    return (v1 + I * v2) / (1 - v3)


ls = []

ls.append(stereo(1 / sqrt(2), 1 / sqrt(2), 0))
ls.append(stereo(1 / sqrt(2), -1 / sqrt(2), 0))
ls.append(stereo(-1 / sqrt(2), 1 / sqrt(2), 0))
ls.append(stereo(-1 / sqrt(2), -1 / sqrt(2), 0))
ls.append(stereo(0, 1 / sqrt(2), 1 / sqrt(2)))
ls.append(stereo(0, 1 / sqrt(2), -1 / sqrt(2)))
ls.append(stereo(0, -1 / sqrt(2), 1 / sqrt(2)))
ls.append(stereo(0, -1 / sqrt(2), -1 / sqrt(2)))
ls.append(stereo(1 / sqrt(2), 0, 1 / sqrt(2)))
ls.append(stereo(1 / sqrt(2), 0, -1 / sqrt(2)))
ls.append(stereo(-1 / sqrt(2), 0, 1 / sqrt(2)))
ls.append(stereo(-1 / sqrt(2), 0, -1 / sqrt(2)))

for x, l in enumerate(ls):
    print(f"ls[{x}]" + " = " + str(l.simplify()))

ls[0] = sqrt(2)*(1 + I)/2
ls[1] = sqrt(2)*(1 - I)/2
ls[2] = sqrt(2)*(-1 + I)/2
ls[3] = sqrt(2)*(-1 - I)/2
ls[4] = I + sqrt(2)*I
ls[5] = -I + sqrt(2)*I
ls[6] = -sqrt(2)*I - I
ls[7] = -sqrt(2)*I + I
ls[8] = 1 + sqrt(2)
ls[9] = -1 + sqrt(2)
ls[10] = -sqrt(2) - 1
ls[11] = 1 - sqrt(2)


The corresponding stereographically-projected points in the complex plane are
given by 

$$

\begin{alignat*}{3}
l_1 &= \tfrac1{\sqrt{2}}\left(1+i\right) \qquad &&l_7 &&= i\left(-1-\sqrt{2}\right)\\
l_2 &= \tfrac1{\sqrt{2}}\left(1-i\right) \qquad&&l_8 &&= i\left(1-\sqrt{2}\right)\\
l_3 &= \tfrac1{\sqrt{2}}\left(-1+i\right) \qquad&&l_9 &&= 1+\sqrt{2}\\
l_4 &= \tfrac1{\sqrt{2}}\left(-1-i\right) \qquad&&l_{10} &&= -1+\sqrt{2}\\
l_5 &= i\left(1+\sqrt{2}\right) \qquad&&l_{11} &&= -1-\sqrt{2}\\
l_6 &= i\left(-1+\sqrt{2}\right) \qquad&&l_{12} &&= 1-\sqrt{2}

\end{alignat*}

$$

Given the above, we define

$$

\chi(u,v) := \prod_{i=1}^{12}\left(u-l_i\,v\right) = u^{12} - 33 u^{8} v^{4} -
33 u^{4} v^{8} + v^{12} \,.

$$

In [14]:
chi = reduce(lambda x, y: x * y, [u - x * v for x in ls])

simplify(chi)

u**12 - 33*u**8*v**4 - 33*u**4*v**8 + v**12

As it turns out, $\chi(u,v)$ is also related to $t(u,v)$ and $W(u,v)$: $\chi$ is
proportional to the *Jacobian* of ${\bf V}(u,v) := (t(u,v), W(u,v))$:

$$

{\textrm Jac}[{\bf V}](u,v) := \det \left[\begin{matrix} \frac{\partial t}{\partial u} & \frac{\partial t}{\partial v} \\[6pt] \frac{\partial W}{\partial u} & \frac{\partial W}{\partial v} \end{matrix}\right] = -8\,\chi(u,v) \,.

$$

In [15]:
Jac = diff(t, u) * diff(W, v) - diff(t, v) * diff(W, u)
simplify(Jac + 8 * chi)

0

Since $\chi \sim t$, and since $t\to-t$ under transformation $III$, we deduce
that $\chi^2$ is an absolute invariant of the octahedral symmetry group. Lastly,
by direct computation we can verify that the absolute invariants $\chi^2$,
$W^3$, and $t^4$ (all sharing the same order of homogeneity) are inter-related:

$$

\chi^2 - W^3 + 108\, t^4 = 0 \,.

$$

In [16]:
simplify(chi**2 - W**3).factor()

-108*u**4*v**4*(u - v)**4*(u + v)**4*(u**2 + v**2)**4

In [17]:
simplify(chi**2 - W**3 + 108 * t**4)

0

## Icosahedral Symmetry



In [97]:
from functools import reduce

# e = exp(2 * pi * I / 5)
e = symbols("e")
n = e**2 + e**3
m = e + e**4

iso_verts = []
for i in range(5):
    iso_verts.append(n * e**i)
    iso_verts.append(m * e**i)

f = u * v * reduce(lambda x, y: x * y, [u - x * v for x in iso_verts])


def e_simp(expr):
    result = Poly(expr, e)
    e_coeffs = reversed(result.all_coeffs())
    simp_coeffs = [0] * 5
    # enforce e**5 = 1
    for i, coeff in enumerate(e_coeffs):
        simp_coeffs[i % 5] += coeff
    # enforce e**4 = - e**3 - e**2 - e - 1
    simp_coeffs = [simp_coeffs[i] - simp_coeffs[4] for i in range(4)]
    return Poly.from_list(reversed(simp_coeffs), e).as_expr()


f = e_simp(f)
simplify(f.expand())


u*v*(u**10 + 11*u**5*v**5 - v**10)